In [2]:
# Model Training
from datasets import load_from_disk

datasets = load_from_disk("./encoded_data4")
datasets

DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet', '__index_level_0__'],
        num_rows: 644
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', '

In [3]:
datasets = datasets.remove_columns(['__index_level_0__'])

In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security on the Internet'],
        num_rows: 644
    })
    valid: Dataset({
        features: ['String', 'Algorithm Design', 'Basic Machine Organisation', 'Computer System', 'Data Manipulation and Analysis', 'Data Organisation and Data Control', 'Elementary Web Authoring', 'Health and Ethical Issues', 'Information Processing', 'Intellectual Property', 'Internet Services and Applications', 'Multimedia Elements', 'Networking and Internet Basics', 'Program Development', 'Spreadsheets and Databases', 'Threats and Security 

In [5]:
model_labels = [label for label in datasets['train'].features.keys() if label not in ['Unnamed: 0', 'String', '__index_level_0__']]

In [6]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [7]:
datasets["train"].column_names

['String',
 'Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [8]:
datasets["train"].features

{'String': Value(dtype='string', id=None),
 'Algorithm Design': Value(dtype='int32', id=None),
 'Basic Machine Organisation': Value(dtype='int32', id=None),
 'Computer System': Value(dtype='int32', id=None),
 'Data Manipulation and Analysis': Value(dtype='int32', id=None),
 'Data Organisation and Data Control': Value(dtype='int32', id=None),
 'Elementary Web Authoring': Value(dtype='int32', id=None),
 'Health and Ethical Issues': Value(dtype='int32', id=None),
 'Information Processing': Value(dtype='int32', id=None),
 'Intellectual Property': Value(dtype='int32', id=None),
 'Internet Services and Applications': Value(dtype='int32', id=None),
 'Multimedia Elements': Value(dtype='int32', id=None),
 'Networking and Internet Basics': Value(dtype='int32', id=None),
 'Program Development': Value(dtype='int32', id=None),
 'Spreadsheets and Databases': Value(dtype='int32', id=None),
 'Threats and Security on the Internet': Value(dtype='int32', id=None)}

In [9]:
#print(f"{datasets.train.column_names}\n{datasets.train.features}")

In [10]:
model_labels

['Algorithm Design',
 'Basic Machine Organisation',
 'Computer System',
 'Data Manipulation and Analysis',
 'Data Organisation and Data Control',
 'Elementary Web Authoring',
 'Health and Ethical Issues',
 'Information Processing',
 'Intellectual Property',
 'Internet Services and Applications',
 'Multimedia Elements',
 'Networking and Internet Basics',
 'Program Development',
 'Spreadsheets and Databases',
 'Threats and Security on the Internet']

In [11]:
id2label = {idx:label for idx, label in enumerate(model_labels)}
label2id = {label:idx for idx, label in enumerate(model_labels)}

In [12]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import numpy as np

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [13]:
def encode_data(dataset):
    text = dataset["String"]
    # tokenize string
    encoding = tokenizer(text, padding="max_length", truncation=True, max_length=70)
    # create encoded array list with labels
    labelsBatch = {x: dataset[x] for x in dataset.keys() if x in model_labels}
    labels_matrix = np.zeros((len(text), len(model_labels)))
    for idx, label in enumerate(model_labels):
        labels_matrix[:, idx] = labelsBatch[label]
    
    encoding["labels"] = labels_matrix.tolist()
    return encoding

In [14]:
# tokenize the dataset
encoded_ds = datasets.map(encode_data, batched=True, remove_columns=datasets['train'].column_names)

Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\train\cache-7700797c4d7f932e.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\valid\cache-ea223c76f3455998.arrow
Loading cached processed dataset at c:\Users\user\Desktop\QuestionQuestionModel\model\encoded_data4\test\cache-459aacbf91ebbdc4.arrow


In [15]:
encoded_ds['train']['labels'][0]

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]

In [16]:
[id2label[idx] for idx, label in enumerate(encoded_ds['train'][15]['labels']) if label == 1.0]

['Data Organisation and Data Control']

In [17]:
import torch
from torch import cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

encoded_ds.set_format("torch")

Using device: cuda



In [18]:
encoded_ds["train"]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 644
})

In [19]:

# define model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", num_labels=len(model_labels), id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classi

In [20]:
# train model
batch_size = 8
metric_name = "f1"

In [21]:
import evaluate
metric  = evaluate.load("accuracy")

In [22]:
import time

train_name_dir = time.strftime(f"%Y%m%d_%H%M%S$-train-model-{model.name_or_path}")
train_name_dir

'20230329_044227$-train-model-distilbert-base-uncased'

In [23]:
from transformers import TrainingArguments, Trainer


args = TrainingArguments(
    output_dir="./distBert-v9/"+ train_name_dir,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs= 81,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='./distBert-v9/logging/'+ train_name_dir
)

In [24]:
args.learning_rate

2e-05

In [25]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)

c:\Users\user\Desktop\QuestionQuestionModel\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, balanced_accuracy_score, hamming_loss
from transformers import EvalPrediction


def multi_label_metrics(predictions, labels, threshold=0.50):
    # apply sigmoid on predictions fitting (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_true.argmax(axis=1), y_pred.argmax(axis=1))
    # logits, labels = predictions
    # prediction = np.argmax(logits, axis=-1)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
               'balanced_accuracy': balanced_accuracy,
               }
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

def compute_metrics_new(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [27]:
encoded_ds['train'][0]['labels'].type()

'torch.FloatTensor'

In [28]:
# model.to(device)

In [29]:
outputs = model(input_ids=encoded_ds['train']['input_ids'][0].unsqueeze(0), labels=encoded_ds['train'][0]['labels'].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7047, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[ 0.0037,  0.0217,  0.1640, -0.0293,  0.0279, -0.0833,  0.0151,  0.0361,
          0.0137,  0.0495,  0.0158,  0.0092, -0.0196, -0.0053,  0.1444]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [30]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_ds["train"],
    eval_dataset=encoded_ds["valid"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

In [31]:
trainer.train()

***** Running training *****
  Num examples = 644
  Num Epochs = 81
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6561
  Number of trainable parameters = 66965007
  1%|          | 81/6561 [00:05<05:03, 21.34it/s] ***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                 
  1%|          | 81/6561 [00:05<05:03, 21.34it/s]Saving model checkpoint to ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-81
Configuration saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-81\config.json


{'eval_loss': 0.28308966755867004, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.1293, 'eval_samples_per_second': 626.686, 'eval_steps_per_second': 85.106, 'epoch': 1.0}


Model weights saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-81\pytorch_model.bin
tokenizer config file saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-81\tokenizer_config.json
Special tokens file saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-81\special_tokens_map.json
  2%|▏         | 162/6561 [00:10<04:59, 21.38it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                  
  2%|▏         | 162/6561 [00:10<04:59, 21.38it/s]Saving model checkpoint to ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-162
Configuration saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-162\config.json


{'eval_loss': 0.2506187856197357, 'eval_f1': 0.0, 'eval_roc_auc': 0.5, 'eval_accuracy': 0.0, 'eval_balanced_accuracy': 0.06666666666666667, 'eval_runtime': 0.1408, 'eval_samples_per_second': 575.269, 'eval_steps_per_second': 78.123, 'epoch': 2.0}


Model weights saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-162\pytorch_model.bin
tokenizer config file saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-162\tokenizer_config.json
Special tokens file saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-162\special_tokens_map.json
  4%|▎         | 243/6561 [00:15<04:54, 21.45it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                  
  4%|▎         | 243/6561 [00:15<04:54, 21.45it/s]Saving model checkpoint to ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-243
Configuration saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-243\config.json


...

100%|█████████▉| 6560/6561 [16:49<00:00, 20.41it/s]***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
                                                   
100%|██████████| 6561/6561 [16:49<00:00, 20.41it/s]Saving model checkpoint to ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-6561
Configuration saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-6561\config.json


{'eval_loss': 0.14079801738262177, 'eval_f1': 0.7926829268292682, 'eval_roc_auc': 0.8721496693925476, 'eval_accuracy': 0.7530864197530864, 'eval_balanced_accuracy': 0.792005772005772, 'eval_runtime': 0.1573, 'eval_samples_per_second': 514.802, 'eval_steps_per_second': 69.911, 'epoch': 81.0}


Model weights saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-6561\pytorch_model.bin
tokenizer config file saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-6561\tokenizer_config.json
Special tokens file saved in ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-6561\special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./distBert-v9/20230329_044227$-train-model-distilbert-base-uncased\checkpoint-4779 (score: 0.8).
100%|██████████| 6561/6561 [17:06<00:00,  6.39it/s]

{'train_runtime': 1026.0982, 'train_samples_per_second': 50.837, 'train_steps_per_second': 6.394, 'train_loss': 0.023440817683296967, 'epoch': 81.0}


TrainOutput(global_step=6561, training_loss=0.023440817683296967, metrics={'train_runtime': 1026.0982, 'train_samples_per_second': 50.837, 'train_steps_per_second': 6.394, 'train_loss': 0.023440817683296967, 'epoch': 81.0})

In [32]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 81
  Batch size = 8
100%|██████████| 11/11 [00:00<00:00, 90.79it/s]


{'eval_loss': 0.13152724504470825,
 'eval_f1': 0.8,
 'eval_roc_auc': 0.8779636228809196,
 'eval_accuracy': 0.7777777777777778,
 'eval_balanced_accuracy': 0.8031168831168831,
 'eval_runtime': 0.1351,
 'eval_samples_per_second': 599.579,
 'eval_steps_per_second': 81.424,
 'epoch': 81.0}

In [33]:
prediction_result = trainer.predict(encoded_ds['test'])

***** Running Prediction *****
  Num examples = 81
  Batch size = 8
100%|██████████| 11/11 [00:00<00:00, 99.19it/s] 


In [34]:
print(prediction_result.metrics)

{'test_loss': 0.16424967348575592, 'test_f1': 0.7878787878787878, 'test_roc_auc': 0.8840574311379794, 'test_accuracy': 0.7530864197530864, 'test_balanced_accuracy': 0.8126695526695527, 'test_runtime': 0.1851, 'test_samples_per_second': 437.515, 'test_steps_per_second': 59.416}


In [35]:
trainer.save_model("./distlBert-model-v9")

Saving model checkpoint to ./distlBert-model-v9
Configuration saved in ./distlBert-model-v9\config.json
Model weights saved in ./distlBert-model-v9\pytorch_model.bin
tokenizer config file saved in ./distlBert-model-v9\tokenizer_config.json
Special tokens file saved in ./distlBert-model-v9\special_tokens_map.json


In [36]:
tokenizer.save_pretrained("./distlBert-model-v9/tokenizer")

tokenizer config file saved in ./distlBert-model-v9/tokenizer\tokenizer_config.json
Special tokens file saved in ./distlBert-model-v9/tokenizer\special_tokens_map.json


('./distlBert-model-v9/tokenizer\\tokenizer_config.json',
 './distlBert-model-v9/tokenizer\\special_tokens_map.json',
 './distlBert-model-v9/tokenizer\\vocab.txt',
 './distlBert-model-v9/tokenizer\\added_tokens.json')